In [2]:
!pip install -U keras-tuner # using colab with this file

     |████████████████████████████████| 61kB 4.7MB/s eta 0:00:011
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=f4153e8951d1959394491f876119d216eef67ff83814a0a586783a6fc901659d
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=b063e8ffcfb1d1d862d78258023b493929bd12e51ab382c1d9881e4297c5bdd8
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch

In [ ]:
batch_size = 128
num_classes = 10
epochs = 12

In [5]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
img_rows, img_cols = 28, 28
train_images = train_images.reshape(len(train_images),img_rows,img_cols,1)
test_images = test_images.reshape(len(test_images),img_rows,img_cols,1)
input_shape = (img_rows, img_cols, 1)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255.0
test_images /= 255.0

In [ ]:
train_labels = to_categorical(train_labels,num_classes)
test_labels = to_categorical(test_labels,num_classes)

In [ ]:
#Use keras tuner to do hyperparameters optimization 
##document from keras tuner 
from kerastuner.engine.hyperparameters import HyperParameters
def build_model(hp):
  model = Sequential()
  model.add(Conv2D(filters=hp.Int('filter1',min_value=32,max_value=512,step=16),
                   kernel_size=hp.Choice('kernel_size1', values = [3,5]),
                   activation='relu',
                   input_shape=input_shape))
  model.add(Conv2D(filters=hp.Int('filter2',min_value=32,max_value=512,step=32),
                   kernel_size=hp.Choice('kernel_size2', values = [3,5]),
                   activation='relu'))
  #model.add(MaxPooling2D(pool_size=(2, 2)))
  #model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),activation='relu'))
  #model.add(Dropout(0.5))
  model.add(Dense(num_classes, activation='softmax'))

  model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),loss='categorical_crossentropy',metrics=['accuracy'])
  return model

In [9]:
tuner = RandomSearch(build_model,objective='val_accuracy',max_trials=5,executions_per_trial=1,directory='my_dir1',project_name='abcd')
tuner.search(train_images, train_labels,epochs=3,validation_split=0.1)
model=tuner.get_best_models(num_models=1)[0]
model.summary()

Epoch 1/3
1688/1688 [==============================] - 16s 9ms/step - loss: 0.5473 - accuracy: 0.8089 - val_loss: 0.4464 - val_accuracy: 0.8362
Epoch 2/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.4045 - accuracy: 0.8494 - val_loss: 0.3702 - val_accuracy: 0.8598
Epoch 3/3
1688/1688 [==============================] - 15s 9ms/step - loss: 0.3735 - accuracy: 0.8612 - val_loss: 0.4001 - val_accuracy: 0.8522


Epoch 1/3
1688/1688 [==============================] - 27s 16ms/step - loss: 0.3780 - accuracy: 0.8644 - val_loss: 0.2977 - val_accuracy: 0.8872
Epoch 2/3
1688/1688 [==============================] - 27s 16ms/step - loss: 0.2367 - accuracy: 0.9136 - val_loss: 0.2541 - val_accuracy: 0.9115
Epoch 3/3
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1738 - accuracy: 0.9355 - val_loss: 0.2428 - val_accuracy: 0.9133


Epoch 1/3
1688/1688 [==============================] - 70s 41ms/step - loss: 1.5142 - accuracy: 0.8285 - val_loss: 0.3650 - val_accuracy: 0.8710
Epoch 2/3
1688/1688 [==============================] - 69s 41ms/step - loss: 0.3459 - accuracy: 0.8727 - val_loss: 0.3731 - val_accuracy: 0.8668
Epoch 3/3
1688/1688 [==============================] - 69s 41ms/step - loss: 0.3138 - accuracy: 0.8827 - val_loss: 0.3830 - val_accuracy: 0.8632


Epoch 1/3
1688/1688 [==============================] - 54s 32ms/step - loss: 1.0111 - accuracy: 0.8243 - val_loss: 0.4011 - val_accuracy: 0.8520
Epoch 2/3
1688/1688 [==============================] - 53s 31ms/step - loss: 0.3666 - accuracy: 0.8644 - val_loss: 0.3733 - val_accuracy: 0.8652
Epoch 3/3
1688/1688 [==============================] - 52s 31ms/step - loss: 0.3359 - accuracy: 0.8751 - val_loss: 0.3738 - val_accuracy: 0.8637


Epoch 1/3
1688/1688 [==============================] - 25s 15ms/step - loss: 0.6225 - accuracy: 0.8310 - val_loss: 0.3826 - val_accuracy: 0.8607
Epoch 2/3
1688/1688 [==============================] - 25s 15ms/step - loss: 0.3534 - accuracy: 0.8723 - val_loss: 0.3624 - val_accuracy: 0.8713
Epoch 3/3
1688/1688 [==============================] - 25s 15ms/step - loss: 0.3143 - accuracy: 0.8842 - val_loss: 0.3454 - val_accuracy: 0.8787


INFO:tensorflow:Oracle triggered exit
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 176)       1760      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 384)       608640    
_________________________________________________________________
flatten (Flatten)            (None, 221184)            0         
_________________________________________________________________
dense (Dense)                (None, 48)                10616880  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 11,227,770
Trainable params: 11,227,770
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 27s 16ms/step - loss: 0.1233 - accuracy: 0.9534 - val_loss: 0.2583 - val_accuracy: 0.9173
Epoch 5/10
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0882 - accuracy: 0.9667 - val_loss: 0.2993 - val_accuracy: 0.9175
Epoch 6/10
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0606 - accuracy: 0.9780 - val_loss: 0.3793 - val_accuracy: 0.9102
Epoch 7/10
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0444 - accuracy: 0.9838 - val_loss: 0.3888 - val_accuracy: 0.9122
Epoch 8/10
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0341 - accuracy: 0.9875 - val_loss: 0.4956 - val_accuracy: 0.9150
Epoch 9/10
1688/1688 [==============================] - 27s 16ms/step - loss: 0.0277 - accuracy: 0.9904 - val_loss: 0.4901 - val_accuracy: 0.9123
Epoch 10/10
1688/1688 [==============================] - 26s 16ms/step - loss: 0.0233 - accuracy: 0.9921 - val_loss: 0.5398 

In [12]:
score = model.evaluate(test_images, test_labels, batch_size=64, verbose=0)
print('Test accuracy:', score[1])

Test accuracy: 0.9074000120162964
